In [1]:
import numpy as np
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
lem = WordNetLemmatizer()
RANDOM_STATE = 50
UNK_ID = 1
PAD_ID = 0
MAX_LEN = 20

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kusha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kusha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kusha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kusha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def clean_string(s):    
    s =  re.sub(r'(?<=[^\s0-9])(?=[.,;?])', r' ', s)
    s = re.sub(r'\((\d+)\)', r'', s)
    s = re.sub(r'\s\s', ' ', s)
    s = re.sub(r"[^A-Za-z0-9(),!?\'`]", " ", s)
    s = re.sub(r"\'s", " \'s", s)
    s = re.sub(r"\'ve", " \'ve", s)
    s = re.sub(r"n\'t", " n\'t", s)
    s = re.sub(r"\'re", " \'re", s)
    s = re.sub(r"\'d", " \'d", s)
    s = re.sub(r"\'ll", " \'ll", s)
    s = re.sub(r",", " , ", s)
    s = re.sub(r"!", " ! ", s)
    s = re.sub(r"\"", " \" ", s)
    s = re.sub(r"\(", " ( ", s)
    s = re.sub(r"\)", " ) ", s)
    s = re.sub(r"\?", " ? ", s)
    s = re.sub(r"\s{2,}", " ", s)
    s = re.sub(r"\.", " . ", s)
    s = re.sub(r"., ", " , ", s)
    s = re.sub(r"\\n", " ", s)
    return s.strip().lower()

def create_train_valid(features,labels,train_fraction = 0.7,max_valid=1000):

    features,labels = shuffle(features,labels,random_state = RANDOM_STATE)

    train_end = max(int(train_fraction*len(labels)),len(labels)-max_valid)

    train_features = np.asarray(features[:train_end])
    valid_features = np.asarray(features[train_end:])

    train_labels = np.asarray(labels[:train_end])
    valid_labels = np.asarray(labels[train_end:])
    
    return train_features,valid_features,train_labels,valid_labels

In [ ]:
pip install transformers

In [ ]:
import time
import torch
import numpy as np
from tqdm import  trange
from transformers import BertModel, BertTokenizer, BertConfig

class BertSentenceEncoder():
    def __init__(self, model_name='bert-base-cased'):
        '''
        Parameters
        ----------
        model_name : string, optional
            DESCRIPTION. The default is 'bert-base-cased'.
            
            Find a list of usable pre-trained bert models from:
                https://huggingface.co/transformers/pretrained_models.html
        '''

        self.model_name =   model_name
        self.config =       BertConfig.from_pretrained(self.model_name, output_hidden_states=True, training=True)
        self.model =        BertModel.from_pretrained(self.model_name, config=self.config)
        self.tokenizer =    BertTokenizer.from_pretrained(self.model_name, do_lower_case=False)
        self.pooling_methods = ['max', 'mean', 'max-mean']
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        # freeze parameters
        self.model.requires_grad_(False)        
        # move model to gpu , if one available:
        if torch.cuda.is_available():
            self.model.cuda()
            
    def __repr__(self):
        return 'BertSentenceEncoder model:{}'.format(self.model_name)
    
    def _mean_pooler(self, encoding):
        return encoding.mean(dim=1)
    
    def _max_pooler(self, encoding):
        return encoding.max(dim=1).values
    
    def _max_mean_pooler(self, encoding):
        return torch.cat((self._max_pooler(encoding), self._mean_pooler(encoding)), dim=1)
    
    def _pooler(self, encodings, pooling_method):
        '''
        Pools the encodings along the time/sequence axis according
        to one of the pooling method:
            - 'max'      :  max value along the sequence/time dimension
                            returns a (batch_size x hidden_size) shaped tensor
            - 'mean'     :  mean of the values along the sequence/time dimension
                            returns a (batch_size x hidden_size) shaped tensor
            - 'max-mean' :  max and mean values along the sequence/time dimension appended
                            returns a (batch_size x 2*hidden_size) shaped tensor
                            [ max : mean ]
        Parameters
        ----------
        encoding : list of tensor to pool along the sequence/time dimension.
        
        pooling_method : one of 'max', 'mean' or 'max-mean'
        
        Returns
        -------
        tensor of shape (batch_size x hidden_size).
        '''
        
        assert (pooling_method in self.pooling_methods), \
            "pooling methods needs to be one of 'max', 'mean' or 'max-mean'"
            
        if pooling_method   == 'max':       pool_fn = self._max_pooler
        elif pooling_method == 'mean':      pool_fn = self._mean_pooler
        elif pooling_method == 'max-mean':  pool_fn = self._max_mean_pooler
        
        pooled = pool_fn(encodings)
        
        return pooled
    

    
    def encoder(self, sentences, layer=-2, pooling_method = None, max_length=40 ):
     
        assert isinstance(sentences, list), \
            "parameter 'sentences' is supposed to be a list of string/s"
        assert all(isinstance(x, str) for x in sentences), \
            "parameter 'sentences' must contain strings only"
        
        '''
        model(input_tokens) returns a tuple of 3 elements.
        out[0] : last_hidden_state  of shape [ B x T x D ]
        out[1] : pooler_output      of shape [ B x D ]
        out[2] : hidden_states      13 tuples, one for each hidden layer
                                    each tuple of shape [ B x T x D ]        
        '''
        with torch.no_grad():
            input_ids = self.tokenizer.batch_encode_plus(sentences, return_tensors='pt', max_length=max_length, pad_to_max_length=True)['input_ids']
            input_ids = input_ids.to(self.device)
            encoded = self.model(input_ids)
                    
        if pooling_method in self.pooling_methods:
            pooled = self._pooler(encoded[2][layer], pooling_method)
            return pooled
        
        return encoded


def get_BE_batched(sentences, batch_size, BE=None):
    assert(BE), "Provide a BertSentenceEncoder object."
    l = len(sentences)
    embeddings = np.empty((0,768))    
    num_batches = int(l/batch_size) if l%batch_size==0 else int(l/batch_size)+1
    
    t = trange(num_batches, desc='Batch', leave=True)

    for i in t:
        # get start and end index for this batch
        if( i != int(l/batch_size) ):
            start   = (i*batch_size)
            end     = (i*batch_size)+batch_size   
        else:
            start   = int(l/batch_size)*batch_size
            end     = l
        t.set_description('Embedding batch => {} : {}'.format(start, end))
    
        s = time.time()
        batch_embeddings = BE.encoder(sentences[start:end], layer = -2, pooling_method='mean')
        e = time.time()    
        print("Time elapsed: {} seconds.".format(e-s))
        batch_embeddings = batch_embeddings.cpu().numpy()
        embeddings = np.append(embeddings, batch_embeddings, axis=0)
        
    return embeddings

In [4]:
import pickle
import numpy as np

with open('../../video_features/resnet_features.pkl', 'rb') as f:
    video_features_dict = pickle.load(f)

for k in video_features_dict:
    video_features_dict[k] = np.mean(video_features_dict[k], axis=0)

video_features_dict[list(video_features_dict.keys())[0]].shape

(2048,)

In [16]:
with open('../../bert embeddings/sarcasm_aug_embeddings.pkl', 'rb') as f:
    text_features_dict = pickle.load(f)

text_features_dict[list(text_features_dict.keys())[0]].shape

(768,)

In [17]:
len(video_features_dict),len(text_features_dict)

(690, 1322)

In [19]:
text_features = []
video_features = []

for i in text_features_dict:
    video_features.append(video_features_dict[i[:-2]])
    text_features.append(text_features_dict[i])

video_features = np.array(video_features)
text_features = np.array(text_features)
video_features.shape, text_features.shape

((1322, 2048), (1322, 768))

In [25]:
data = pd.read_csv('../../datasets/mustard_dataset/sarcasm_with_id.csv')
idToLabel = {}

ids = list(data['id'])
lab = [int(k) for k in list(data['sarcasm'])]

iToL = {}
for i in range(len(ids)):
    iToL[ids[i]] = lab[i]

labels = []
for i in text_features_dict:
    labels.append(iToL[i])

len(labels)

1322

In [26]:
def svm_train(features,labels):
    clf = make_pipeline(
        StandardScaler(),
        svm.SVC(C=15.0, gamma="scale", kernel="rbf")
    )
    return clf.fit(features, labels)

In [27]:
def svm_test(clf,features,labels):
    pred = clf.predict(features)
    true = labels

    result_string = classification_report(true, pred, digits=3)
    print(confusion_matrix(true, pred))
    print(result_string)

In [28]:
final_features = np.concatenate((video_features, text_features), axis=1)
final_features.shape

(1322, 2816)

In [29]:
x_train, x_valid, y_train, y_valid = create_train_valid(final_features,labels)

In [30]:
clf = svm_train(x_train,y_train)

svm_test(clf,x_valid,y_valid)

[[194  12]
 [ 22 169]]
              precision    recall  f1-score   support

           0      0.898     0.942     0.919       206
           1      0.934     0.885     0.909       191

    accuracy                          0.914       397
   macro avg      0.916     0.913     0.914       397
weighted avg      0.915     0.914     0.914       397

